In [ ]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
from sklearn.datasets import make_classification

In [ ]:
#Start off by creating some toy data. 
X, y = make_classification(10000, n_features=2, n_redundant=0, n_classes=4,
                       n_clusters_per_class=1)
# Convert to theano floatX.
X = X.astype(theano.config.floatX)
# Labels should be ints.
y = y.astype('int32')
X_train, X_val, X_test = X[:7500], X[7500:8000], X[8000:]
y_train, y_val, y_test = y[:7500], y[7500:8000], y[8000:]

In [ ]:
#Create a multilayer perceptron with mini-batch GD using theano and lasagne.

def basic_mlp(X_train, y_train, X_val, y_val, X_test, hidden_units=40,
              n_classes=4, batch_size=100):    
    #First, construct an input layer.
    #The shape parameter defines the expected input shape,
    #which is just the shape of our data matrix X_train.
    l_in = lasagne.layers.InputLayer(shape=X_train.shape)

    #We'll create a network with two dense layers.
    #One hidden layer and one output layer.
    #hidden units are set arbitarily by the user
    #in the output layer num_units should equal the number of classes.
    l_hidden = lasagne.layers.DenseLayer(
        l_in,
        num_units=hidden_units,
        nonlinearity=lasagne.nonlinearities.rectify)

    l_output = lasagne.layers.DenseLayer(
        l_hidden, num_units=n_classes,
        nonlinearity=lasagne.nonlinearities.softmax)

    #To actually compute the Theano expression of a stack of Layer subclass
    #instances use the get_output function.
    net_output = lasagne.layers.get_output(l_output)

    #create a loss function.
    true_output = T.ivector('true_output')
    loss = T.mean(lasagne.objectives.categorical_crossentropy(net_output,
                                                              true_output))
    #generate parameter updates.
    all_params = lasagne.layers.get_all_params(l_output)
    updates = lasagne.updates.sgd(loss, all_params, learning_rate=1)
    
    #compile Theano functions for training and computing the output.
    train = theano.function([l_in.input_var, true_output], loss, updates=updates)
    get_output = theano.function([l_in.input_var], net_output)      
    
    n_epochs = 10
    batch_id = 0
    epoch = 0

    while epoch < n_epochs:
        #extract the training data for X and y.
        batch_train_X = X_train[batch_id:batch_id + batch_size]
        batch_train_y = y_train[batch_id:batch_id + batch_size]
        #update the parameters using the extracted data.
        train(batch_train_X, batch_train_y)
        batch_id += batch_size
        #once we've trained on the entire dataset we can check the accuarcy
        #of the model against the validation set.

        if batch_id >= X_train.shape[0]:
            #reset batch index.
            batch_id = 0
            #update the number of epochs.
            epoch += 1
            #compute the network's output on the validation data.
            val_output = get_output(X_val)
            #the predicted class is the index of the largest probability
            #in the output.
            val_predictions = np.argmax(val_output, axis=1)
            #the accuarcy is the average number of correct predictions.
            accuracy = np.mean(val_predictions==y_val)
            print("Epoch {} validation accuracy: {}".format(epoch, accuracy))
               
    test_size = len(X_test)
    batch_id = 0
    results = []

    while batch_id < test_size:
        #extract the test data
        batch_test_x = X_test[batch_id:batch_id + batch_size]
        #compute the network's output on the test data
        test_output = get_output(batch_test_x)
        #the predicted class is the index with the largest probability in the
        #output.
        test_predictions = np.argmax(test_output, axis=1)
        #append the results to the list above.
        results.extend(test_predictions)
        #increase the batch_id
        batch_id += batch_size
        
    #save the predictions as a csv using numpy's savetxt.
    np.savetxt("numpy_test.csv", results, delimiter=",", fmt="%2i")

In [ ]:
def check_prediction_accuracy(y_test):
    #load the predictions made in the function above using numpy's genfromtxt.
    preds = np.genfromtxt("numpy_test.csv")
    accuracy = np.mean(preds==y_test)
    return accuracy